In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json

from trijunction.gate_design import gate_coords
from trijunction.constants import scale, bands, topological_gap, sides
import trijunction.trijunction as trijunction
import trijunction.optimization as optimization
import trijunction.parameters as parameters
from trijunction.tools import hamiltonian
from trijunction.utils import eigsh, svd_transformation, dict_update, dep_acc_index, order_wavefunctions, density

In [89]:
import kwant

In [85]:
import trijunction.utils
import importlib
importlib.reload(trijunction.utils)

<module 'trijunction.utils' from '/home/tinkerer/Trijunction/trijunction/utils.py'>

#### Import configuration file

In [11]:
with open(os.path.realpath('config.json'), 'r') as f:
    config = json.load(f)

change_config = [
    {"device":
     {"grid_spacing":
      {"twoDEG": 0.5, "normal": 1, "dielectric": 1, "air": 5, "gate": 1, "substrate": 1}}},
    {"gate": {"channel_width": 13.0, "L":40, 'angle': np.pi/6, 'gap': 4}},
]

for local_config in change_config:
    config = dict_update(config, local_config)

#### Create an instance of Trijunction class with the predefined or new configuration

In [14]:
%%capture output
%%time
system = trijunction.Trijunction(config, optimize_phase_pairs=[])

#### Look at the discretized trijunction

In [44]:
sites = np.array([site.pos for site in system.trijunction.sites])/scale
gates = system.poisson_system.regions.voltage.tag_points
gate_names = np.array(list(gates.keys()))
depletion_gates = gate_names[np.logical_not([gate.startswith(('global', 'dirichlet')) for gate in gate_names])]
full_grid = system.grid_points 

In [ ]:
plt.figure(figsize=(5, 5))
CB_friendly_color = ['#ff7f00', '#e41a1c',
                    ] #https://gist.github.com/thriveth/8560036

plt.scatter(sites[:, 0], sites[:, 1], s = 0.1, c = CB_friendly_color[0])
for gate in depletion_gates:
    gate_sites = full_grid[gates[gate]]
    plt.scatter(gate_sites[:, 0], gate_sites[:, 1], s = 0.5, c = CB_friendly_color[1])
plt.axis('off');
plt.savefig('figures/trijunction.png', format='png')

#### Plot Majorana wavefunctions in the nanowire

In [ ]:
voltages = parameters.voltage_dict([-7e-3, -6.8e-3, -7e-3, -7e-3])
summed_ham = sum(
    [
        voltages[key] * system.linear_terms[key]
        for key, value in system.linear_terms.items()
    ]
)
numerical_hamiltonian = system.base_ham + summed_ham


eigval, eigvec = trijunction.utils.eigsh(
    numerical_hamiltonian, 6, sigma=0, return_eigenvectors=True
)

fig, ax = plt.subplots(1, 3)
for i in range(3):
    kwant.plotter.density(system.trijunction, 
                          trijunction.utils.density(eigvec.T[i]),
                          ax=ax[i],
                          colorbar=False);
plt.savefig("figures/Decoupled_wavefunctions.png", format='png')

### Run optimization routine
Note: It takes about 30minutes or so in a single core depending on the specified configuration. Hence please 
save your results often.

In [131]:
# Import parameters

pair = 'left-right'
params = parameters.junction_parameters()
params.update(potential=system.flat_potential())

index = system.indices.copy()

# Set-up for specific pair to be optimized
# remove 50% of the points from the channel to be depleted that is closest to the center.
depleted_channel = list(set(sides)-set(pair.split('-')))[0]
depleted_indices = index[depleted_channel]
index[depleted_channel]  = depleted_indices[:int(len(depleted_indices)*50/100)]

params['dep_acc_index'] = index

initial_condition = (-3e-3, -3e-3, -3e-3, 10e-3)

##### Step1: Optimize potential shape

In [ ]:
args = (pair.split('-'),
        (system.base_ham, system.linear_terms),
        params['dep_acc_index'], 
        )

sol1 = minimize(shape_loss, 
         x0=initial_condition, 
         args=args, 
         method='trust-constr', 
         options={'initial_tr_radius':1e-3}
        )

##### Step2: Optimize wavefunctions

In [ ]:
weights = [1, 1, 1e1]
args = (
    pair.split('-'),
    (system.base_ham, 
     system.linear_terms,
     system.mlwf[order_wavefunctions(pair)]),
    params['dep_acc_index'],
    weights
)


sol2 = minimize(wavefunction_loss, 
     x0=sol1.x, 
     args=args, 
     method='trust-constr',
     options={
         'initial_tr_radius': 1e-3
     }
    )

##### Step3: Optimize phase

In [ ]:
initial_condition = sol2.x

params.update(parameters.voltage_dict(initial_condition))

args = (pair, 
        params, 
        (system.trijunction, 
         system.linear_terms, 
         system.f_params,
         system.mlwf[order_wavefunctions(pair)]
        )
       )

sol3 = minimize_scalar(loss,
                        args=args, 
                        method='bounded',
                        bounds=(0,2),
                      )


phase = sol3.x

##### Optimize Majorana couplings

In [ ]:
params.update(parameters.phase_pairs(pair, phase*np.pi))

base_ham = system.trijunction.hamiltonian_submatrix(
    sparse=True, params=system.f_params(**params)
)

args = (pair, 
        params, 
        (base_ham, system.linear_terms, 
         None,
         system.mlwf[order_wavefunctions(pair)]
        ),
       )

sol4 = minimize(loss, 
         x0=sol2.x, 
         args=args, 
         method='trust-constr', 
         options={
             'initial_tr_radius':1e-3,
         }
        )